# **ANÁLISE DE REDES**
## Atividade 01 - Prática com PyVis e Streamlit
# **Autores**
### José Alex
### Fabricio Cruz

## **Instalando pyvis**

In [ ]:
!pip install pyvis

## **Importando bibliotecas e dataset**
### **Sobre o dataset:** O OpenFlights/Airline Route Mapper Route Database mapeou 59.036 rotas entre 3.209 aeroportos em 531 companhias aéreas em todo o mundo em janeiro de 2012. Segue aqui o link do dataset disponível no Kaggle: <https://www.kaggle.com/datasets/open-flights/flight-route-database>
### O nosso dataset não é fortemente conectado. Logo, não conseguimos encontrar valores de diâmetro e periferia. Para explorar essas funções vamos usar o dataset **NetworkOfThrones** que é um grafo não direcionado conectado.

In [83]:
from pyvis.network import Network
import networkx as nx
import pandas as pd
from IPython.core.display import display, HTML
import scipy as sp
import matplotlib.pyplot as plt
import collections

got_data = pd.read_csv("https://raw.githubusercontent.com/pupimvictor/NetworkOfThrones/master/stormofswords.csv")
routes_data = pd.read_csv("routes.csv")

G = nx.from_pandas_edgelist(got_data, source = "Source", target = "Target")
R = nx.from_pandas_edgelist(routes_data, source = "Source", target = "Target", create_using=nx.DiGraph())